In [1]:
import pandas as pd
import json
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from pathlib import Path
import scipy.sparse as sparse
from lightfm import LightFM
from lightfm.cross_validation import random_train_test_split
from lightfm.evaluation import precision_at_k, recall_at_k
from lightfm.data import Dataset

In [2]:
df_sub = pd.read_csv(Path.cwd() / 'data' / 'sample_submission.csv')
df_train = pd.read_csv(Path.cwd() / 'data' / 'train.csv')
df_test = pd.read_csv(Path.cwd() / 'data' / 'test.csv')

/home/alex/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3071: DtypeWarning: Columns (8) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
/home/alex/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3071: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
df_train.drop_duplicates(inplace=True)

In [4]:
df_train[(df_train.userid == 538)].rating.value_counts()

0.0    46
1.0    43
Name: rating, dtype: int64

In [57]:
df_test.userid.max()

127495

In [3]:
df_train_train, df_train_test = train_test_split(df_train, random_state=32, shuffle=True)

In [4]:
ratings_coo = sparse.coo_matrix((df_train['rating'].astype(int),
                                 (df_train['userid'],
                                  df_train['itemid'])))

In [5]:
ratings_coo_train = sparse.coo_matrix((df_train_train['rating'].astype(int),
                                 (df_train_train['userid'],
                                  df_train_train['itemid'])))

In [6]:
ratings_coo.row

array([102179,   3625,  39495, ...,     88,  41540,    466], dtype=int32)

In [28]:
df_train.rating.value_counts()

1.0    703165
0.0    125770
Name: rating, dtype: int64

In [18]:
len(df_train['overall'].astype(int))

828935

In [8]:
len(df_train)

828935

In [9]:
#Ошибку не дает, но пока непонятно как правильно сформировать и подцепить в модель
#Просто метод формирования матрицы признаков на будущее.
user_feature_coo = sparse.coo_matrix((df_train['overall'].astype(int),
                                     (df_train['userid'],
                                     [0]*len(df_train))))

In [61]:
#Тест тест с датасет
dataset = Dataset()

dataset.fit(
            users=range(df_train.userid.max()+1),
            items=range(df_train.itemid.max()+1) #,
          #  user_features=['overall:1', 'overall:2', 'overall:3', 'overall:4', 'overall:5'],
          #  item_features=[]
)

(interactions, weights) = dataset.build_interactions(
    [(x[11], x[12], x[13]) for x in df_train.values])

In [12]:
user_feature_coo.shape

(127496, 1)

In [63]:
ratings_coo.shape

(127496, 41320)

In [202]:
NUM_THREADS = 8 #число потоков
NUM_COMPONENTS = 180 #число параметров вектора 
NUM_EPOCHS = 14 #число эпох обучения
LEARNING_RATE = 0.13
RANDOM_STATE = 42

model = LightFM(learning_rate=LEARNING_RATE, loss='logistic', learning_schedule='adagrad',
                no_components=NUM_COMPONENTS, random_state=RANDOM_STATE)
model = model.fit(ratings_coo_train,
                  epochs=NUM_EPOCHS, 
                  num_threads=NUM_THREADS)

In [203]:
preds = model.predict(df_train_test.userid.values,
                      df_train_test.itemid.values,
                     )

In [204]:
roc_auc_score(df_train_test.rating, preds)

0.7537410211657183

In [205]:
model_sub = LightFM(learning_rate=LEARNING_RATE, loss='logistic', learning_schedule='adagrad',
                no_components=NUM_COMPONENTS, random_state=RANDOM_STATE)
model_sub = model_sub.fit(ratings_coo,
                  epochs=NUM_EPOCHS, 
                  num_threads=NUM_THREADS)
preds_sub = model_sub.predict(df_test.userid.values,
                      df_test.itemid.values)

In [206]:
# тест:  ROC-AUC должен быть близок к 1
roc_auc_score(df_train_test.rating, 
                model_sub.predict(df_train_test.userid.values, df_train_test.itemid.values))

0.9973893865020431

In [207]:
preds_sub.min(), preds_sub.max()

(-33.010276794433594, 31.718379974365234)

In [208]:
normalized_preds_sub = (preds_sub - preds_sub.min())/(preds_sub - preds_sub.min()).max()

In [209]:
normalized_preds_sub.min(), normalized_preds_sub.max()

(0.0, 1.0)

In [210]:
df_sub['rating']= normalized_preds_sub
df_sub.to_csv('submission_base_log_14.csv', index=False)

'''
End of PART ONE
'''

'\nEnd of PART ONE\n'

In [200]:
'''
PART TWO
'''

NUM_THREADS_2 = 8 #число потоков
NUM_COMPONENTS_2 = 180 #число параметров вектора 
NUM_EPOCHS_2 = 14 #число эпох обучения
RANDOM_STATE_2 = 42
LEARNING_RATE_2 = 0.13

model2 = LightFM(learning_rate=LEARNING_RATE_2, loss='logistic', learning_schedule='adagrad',
                no_components=NUM_COMPONENTS_2, random_state=RANDOM_STATE_2)
model2 = model2.fit(ratings_coo_train,
                  epochs=NUM_EPOCHS_2, 
                  num_threads=NUM_THREADS_2)

In [201]:
preds_2 = model2.predict(df_train_test.userid.values, df_train_test.itemid.values)
roc_auc_score(df_train_test.rating, preds_2)

0.7537207352141249

In [11]:
model_sub2 = LightFM(learning_rate=LEARNING_RATE, loss='logistic', learning_schedule='adagrad',
                no_components=NUM_COMPONENTS, random_state=RANDOM_STATE)
model_sub2 = model2.fit(ratings_coo,
                  epochs=NUM_EPOCHS, 
                  num_threads=NUM_THREADS)
preds_sub2 = model_sub2.predict(df_test.userid.values,
                      df_test.itemid.values)

In [12]:
roc_auc_score(df_train_test.rating, preds)

0.9912157543145635

In [20]:
preds_sub.min(), preds_sub.max()

(-33.86893844604492, 31.111011505126953)

In [21]:
normalized_preds = (preds_sub - preds_sub.min())/(preds_sub - preds_sub.min()).max()

In [22]:
normalized_preds.min(), normalized_preds.max()

(0.0, 1.0)

In [23]:
df_sub['rating']= normalized_preds

In [24]:
df_sub.to_csv('submission_base_log_4.csv', index=False)